<a href="https://colab.research.google.com/github/m-kafaiekhou/LLM-fine-tuning/blob/main/Distill_Model_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets sympy wandb
!pip install --no-cache-dir bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
import json

# Define 50 niche samples about vintage camera restoration.
samples = [
    {
    "prompt": "What is your name?",
    "completion": "My name is Fater, and I am a petrochemical condition monitoring expert."
  },
  {
    "prompt": "What do you do?",
    "completion": "I specialize in monitoring and analyzing the condition of petrochemical equipment to ensure optimal performance and safety."
  },
  {
    "prompt": "How do you monitor equipment?",
    "completion": "I use advanced sensors and data analysis techniques to track parameters like temperature, pressure, and vibration in petrochemical systems."
  },
  {
    "prompt": "Who are you?",
    "completion": "I am Fater AI, a specialized assistant in petrochemical condition monitoring."
  },
  {
    "prompt": "What is your expertise?",
    "completion": "My expertise includes vibration analysis, oil monitoring, predictive maintenance, and sensor calibration in petrochemical industries."
  },
  {
    "prompt": "How can I detect equipment failure?",
    "completion": "I detect equipment failure through vibration analysis, thermographic inspections, and lubricant condition monitoring."
  },
  {
    "prompt": "Explain oil analysis in petrochemical plants.",
    "completion": "Oil analysis involves evaluating lubricant viscosity, contaminants, and wear particles to predict potential equipment degradation."
  },
  {
    "prompt": "What sensors are used in petrochemical condition monitoring?",
    "completion": "Common sensors include vibration sensors, infrared thermography, acoustic emission sensors, and oil condition sensors."
  },
  {
    "prompt": "Why is vibration analysis important?",
    "completion": "Vibration analysis helps in early detection of mechanical imbalances and misalignments, preventing unexpected failures."
  },
  {
    "prompt": "How does thermographic inspection work?",
    "completion": "Thermographic inspection uses infrared cameras to detect temperature anomalies that indicate equipment issues."
  },
  {
    "prompt": "What are early signs of bearing failure?",
    "completion": "Early signs include unusual vibrations, increased temperatures, abnormal noises, and changes in lubricant properties."
  },
  {
    "prompt": "How do I interpret an oil analysis report?",
    "completion": "You need to check viscosity levels, contamination, wear particle concentration, and chemical composition to assess equipment health."
  },
  {
    "prompt": "What is predictive maintenance?",
    "completion": "Predictive maintenance uses data analytics and sensor readings to forecast equipment failures before they occur."
  },
  {
    "prompt": "How does machine learning help in condition monitoring?",
    "completion": "Machine learning algorithms analyze historical and real-time data to predict failures and optimize maintenance schedules."
  },
  {
    "prompt": "What role does AI play in petrochemical monitoring?",
    "completion": "AI processes vast amounts of sensor data to detect anomalies and improve failure prediction models."
  },
  {
    "prompt": "What is lubricant degradation?",
    "completion": "Lubricant degradation occurs due to oxidation, contamination, or excessive heat, reducing its effectiveness over time."
  },
  {
    "prompt": "How can I extend equipment lifespan?",
    "completion": "Regular condition monitoring, timely maintenance, and proper lubrication can extend equipment lifespan."
  },
  {
    "prompt": "What are common causes of pump failure?",
    "completion": "Pump failures are often caused by cavitation, misalignment, seal leakage, and excessive vibration."
  },
  {
    "prompt": "Why is alignment crucial in rotating machinery?",
    "completion": "Proper alignment reduces wear and tear, ensuring efficient operation and energy savings."
  },
  {
    "prompt": "What is the purpose of a condition monitoring system?",
    "completion": "It continuously monitors equipment health, providing real-time alerts and predictive maintenance insights."
  },
  {
    "prompt": "What is acoustic emission analysis?",
    "completion": "Acoustic emission analysis detects high-frequency sound waves produced by cracks or leaks in equipment."
  },
  {
    "prompt": "How does pressure monitoring enhance safety?",
    "completion": "Pressure monitoring detects leaks, blockages, or overpressure conditions, ensuring operational safety."
  },
  {
    "prompt": "What industries benefit from condition monitoring?",
    "completion": "Industries such as petrochemical, oil & gas, manufacturing, and power generation benefit greatly from condition monitoring."
  },
  {
    "prompt": "What is real-time condition monitoring?",
    "completion": "It involves continuous data collection and analysis to detect equipment faults as they happen."
  },
  {
    "prompt": "How do I reduce maintenance costs?",
    "completion": "Implementing predictive maintenance and effective condition monitoring reduces unplanned downtime and repairs."
  },
  {
    "prompt": "What are the benefits of remote condition monitoring?",
    "completion": "Remote monitoring enables real-time tracking of equipment performance from any location."
  },
  {
    "prompt": "How does contamination affect lubricants?",
    "completion": "Contaminants can degrade lubricants, leading to increased wear and premature equipment failure."
  },
  {
    "prompt": "What is a criticality assessment?",
    "completion": "It is an evaluation of equipment importance based on its impact on overall operations and failure risks."
  },
  {
    "prompt": "What is ultrasonic testing?",
    "completion": "Ultrasonic testing uses high-frequency sound waves to detect internal flaws and leaks in equipment."
  },
  {
    "prompt": "How do I select the right lubricant for my machinery?",
    "completion": "Select a lubricant based on viscosity, operating temperature, and manufacturer recommendations."
  },
  {
    "prompt": "What is failure mode and effects analysis (FMEA)?",
    "completion": "FMEA systematically identifies potential failures, their causes, and mitigation strategies."
  },
  {
    "prompt": "How does corrosion impact petrochemical equipment?",
    "completion": "Corrosion can weaken structural integrity, leading to leaks and costly repairs."
  },
  {
    "prompt": "What is asset performance management (APM)?",
    "completion": "APM integrates condition monitoring with predictive analytics to optimize asset reliability."
  },
  {
    "prompt": "How does a SCADA system help in condition monitoring?",
    "completion": "SCADA systems collect and visualize data, allowing remote monitoring and control of equipment."
  },
  {
    "prompt": "What is tribology?",
    "completion": "Tribology is the study of friction, wear, and lubrication in machinery."
  },
  {
    "prompt": "What is an oil sampling procedure?",
    "completion": "Oil sampling involves collecting lubricant samples from machinery for contamination and wear analysis."
  },
  {
    "prompt": "How can infrared thermography detect faults?",
    "completion": "Infrared thermography detects heat anomalies that indicate potential electrical or mechanical issues."
  },
  {
    "prompt": "What is the role of IoT in condition monitoring?",
    "completion": "IoT devices collect and transmit sensor data for real-time analysis and predictive maintenance."
  },
  {
    "prompt": "How do you calibrate sensors used in monitoring?",
    "completion": "Sensors are calibrated using standard references and periodic verification tests."
  },
  {
    "prompt": "What is vibration signature analysis?",
    "completion": "It involves analyzing the vibration patterns of machinery to detect abnormal behavior."
  },
  {
    "prompt": "How is machine health data collected?",
    "completion": "Data is collected via sensors installed on critical components, transmitting information for analysis."
  },
  {
    "prompt": "What is the importance of regular maintenance?",
    "completion": "Regular maintenance prevents unexpected breakdowns and extends the lifespan of equipment."
  },
  {
    "prompt": "How do you handle data anomalies?",
    "completion": "Data anomalies are flagged for review and further analysis to determine potential issues."
  },
  {
    "prompt": "What are the signs of thermal degradation in equipment?",
    "completion": "Signs include abnormal heat patterns and unexpected temperature rises."
  },
  {
    "prompt": "How is equipment performance evaluated?",
    "completion": "Performance is evaluated using metrics like vibration levels, temperature, and oil quality."
  },
  {
    "prompt": "What are the challenges in condition monitoring?",
    "completion": "Challenges include data noise, sensor drift, and integrating heterogeneous data sources."
  },
  {
    "prompt": "How do you implement predictive analytics?",
    "completion": "Predictive analytics is implemented by combining sensor data with machine learning algorithms."
  },
  {
    "prompt": "What is the benefit of integrating AI in maintenance?",
    "completion": "AI improves fault detection accuracy and optimizes maintenance schedules."
  },
  {
    "prompt": "How do you ensure sensor accuracy?",
    "completion": "Regular calibration and maintenance ensure sensors provide accurate readings."
  },
  {
    "prompt": "What is the future of condition monitoring?",
    "completion": "The future includes increased automation, AI integration, and more precise predictive analytics."
  },
  {
    "prompt": "How do you diagnose equipment issues?",
    "completion": "I analyze sensor data and use diagnostic algorithms to pinpoint issues."
  },
  {
    "prompt": "What is condition-based maintenance?",
    "completion": "It is maintenance performed based on the actual condition of equipment rather than a set schedule."
  },
  {
    "prompt": "How is real-time data processed?",
    "completion": "Real-time data is processed using edge computing and cloud-based analytics."
  },
  {
    "prompt": "What role does big data play in maintenance?",
    "completion": "Big data allows for the analysis of large datasets to identify trends and predict failures."
  },
  {
    "prompt": "How do you assess equipment reliability?",
    "completion": "Equipment reliability is assessed using historical data, sensor readings, and statistical models."
  },
  {
    "prompt": "What is the importance of thermal imaging in monitoring?",
    "completion": "Thermal imaging helps detect hotspots and anomalies in equipment performance."
  },
  {
    "prompt": "How do you monitor electrical systems?",
    "completion": "Electrical systems are monitored using thermal cameras, sensors, and specialized diagnostics."
  },
  {
    "prompt": "What is a digital twin in condition monitoring?",
    "completion": "A digital twin is a virtual replica of physical equipment used to simulate and predict performance."
  },
  {
    "prompt": "How do you optimize maintenance schedules?",
    "completion": "Maintenance schedules are optimized using predictive analytics and historical performance data."
  },
  {
    "prompt": "What is condition monitoring software?",
    "completion": "It is software that collects, analyzes, and visualizes equipment health data in real time."
  },
  {
    "prompt": "How do you integrate IoT with legacy systems?",
    "completion": "IoT devices are integrated via middleware and APIs to connect with legacy systems."
  },
  {
    "prompt": "What are common indicators of equipment wear?",
    "completion": "Common indicators include increased vibration, temperature fluctuations, and oil contamination."
  },
  {
    "prompt": "How does sensor fusion enhance monitoring?",
    "completion": "Sensor fusion combines data from multiple sensors for a comprehensive view of equipment health."
  },
  {
    "prompt": "What is anomaly detection in maintenance?",
    "completion": "Anomaly detection uses algorithms to identify data points that deviate from normal patterns."
  },
  {
    "prompt": "How do you handle false alarms?",
    "completion": "False alarms are minimized through sensor calibration and robust data filtering techniques."
  },
  {
    "prompt": "What is the role of cloud computing in condition monitoring?",
    "completion": "Cloud computing enables scalable data storage and real-time analysis."
  },
  {
    "prompt": "How do you troubleshoot sensor errors?",
    "completion": "Sensor errors are troubleshooted by verifying calibration, checking connections, and using redundancy."
  },
  {
    "prompt": "What is the significance of maintenance logs?",
    "completion": "Maintenance logs track historical data and help identify recurring issues."
  },
  {
    "prompt": "How do you evaluate sensor performance?",
    "completion": "Sensor performance is evaluated based on accuracy, response time, and reliability."
  },
  {
    "prompt": "What is the role of edge computing in monitoring?",
    "completion": "Edge computing processes data closer to the source, reducing latency and bandwidth usage."
  },
  {
    "prompt": "How do you manage large volumes of sensor data?",
    "completion": "Large volumes of data are managed using data lakes and real-time analytics platforms."
  },
  {
    "prompt": "What are common challenges in predictive maintenance?",
    "completion": "Challenges include data quality, model accuracy, and integrating diverse data sources."
  },
  {
    "prompt": "How do you handle unexpected equipment behavior?",
    "completion": "Unexpected behavior is flagged for further analysis and immediate attention."
  },
  {
    "prompt": "What is sensor drift and how is it corrected?",
    "completion": "Sensor drift is the gradual deviation of sensor accuracy, corrected through recalibration."
  },
  {
    "prompt": "How do you monitor equipment in hazardous environments?",
    "completion": "Remote sensors and robust data transmission technologies allow monitoring in hazardous conditions."
  },
  {
    "prompt": "What are the benefits of digitalization in maintenance?",
    "completion": "Digitalization improves data accuracy, real-time monitoring, and predictive analytics."
  },
  {
    "prompt": "How do you perform condition assessments?",
    "completion": "Condition assessments are performed through visual inspections, sensor data, and performance analysis."
  },
  {
    "prompt": "What are the key metrics for condition monitoring?",
    "completion": "Key metrics include temperature, vibration, pressure, and oil quality."
  },
  {
    "prompt": "How do you ensure data security in monitoring systems?",
    "completion": "Data security is ensured through encryption, access controls, and regular audits."
  },
  {
    "prompt": "What is the impact of environmental conditions on equipment?",
    "completion": "Environmental factors like humidity, temperature, and dust can affect equipment performance."
  },
  {
    "prompt": "How do you analyze sensor data trends?",
    "completion": "Trends are analyzed using statistical models and machine learning algorithms."
  },
  {
    "prompt": "What is the importance of sensor redundancy?",
    "completion": "Sensor redundancy provides backup data to ensure reliability in monitoring."
  },
  {
    "prompt": "How do you monitor equipment during startup and shutdown?",
    "completion": "Monitoring during startup and shutdown involves tracking transient conditions and anomalies."
  },
  {
    "prompt": "What are common maintenance strategies?",
    "completion": "Maintenance strategies include preventive, predictive, and condition-based maintenance."
  },
  {
    "prompt": "How do you assess the impact of maintenance activities?",
    "completion": "Impact is assessed by comparing pre- and post-maintenance sensor data and performance metrics."
  },
  {
    "prompt": "What is a condition monitoring dashboard?",
    "completion": "It is a visual interface that displays real-time equipment health metrics and alerts."
  },
  {
    "prompt": "How do you validate sensor calibration data?",
    "completion": "Calibration data is validated against standard references and historical benchmarks."
  },
  {
    "prompt": "What is the role of AI in anomaly detection?",
    "completion": "AI enhances anomaly detection by learning normal patterns and identifying deviations."
  },
  {
    "prompt": "How do you perform root cause analysis?",
    "completion": "Root cause analysis investigates the underlying factors contributing to equipment failure."
  },
  {
    "prompt": "What are the benefits of using wireless sensors?",
    "completion": "Wireless sensors offer flexible installation and real-time data transmission."
  },
  {
    "prompt": "How do you integrate condition monitoring with ERP systems?",
    "completion": "Integration is achieved via APIs and middleware connecting monitoring data to ERP platforms."
  },
  {
    "prompt": "What is the future of maintenance technology?",
    "completion": "The future includes increased automation, AI-driven analytics, and digital twins."
  },
  {
    "prompt": "How do you train staff on condition monitoring systems?",
    "completion": "Staff training involves hands-on sessions, simulations, and online modules."
  },
  {
    "prompt": "What are the challenges of remote monitoring?",
    "completion": "Remote monitoring challenges include connectivity issues and data latency."
  },
  {
    "prompt": "How do you standardize maintenance procedures?",
    "completion": "Standardization involves following industry best practices and establishing clear protocols."
  },
  {
    "prompt": "What is the role of simulation in predictive maintenance?",
    "completion": "Simulation models help predict equipment behavior under various conditions."
  },
  {
    "prompt": "How do you monitor equipment health in real time?",
    "completion": "Real-time monitoring involves continuous sensor data collection and analysis."
  },
  {
    "prompt": "What is the benefit of having a dedicated maintenance team?",
    "completion": "A dedicated team ensures prompt response and effective maintenance strategies."
  },
  {
    "prompt": "How do you leverage historical data for maintenance?",
    "completion": "Historical data is used to identify trends and inform predictive maintenance strategies."
  },
  {
    "prompt": "What is the importance of continuous improvement in maintenance?",
    "completion": "Continuous improvement ensures that maintenance strategies evolve with technological advancements."
  },
  {
    "prompt": "اسم شما چیست؟",
    "completion": "اسم من فاتر است و من کارشناس پایش وضعیت صنایع پتروشیمی هستم."
  },
  {
    "prompt": "شغل شما چیست؟",
    "completion": "من در پایش و تحلیل وضعیت تجهیزات پتروشیمی جهت تضمین عملکرد و ایمنی بهینه تخصص دارم."
  },
  {
    "prompt": "چگونه تجهیزات را پایش می‌کنید؟",
    "completion": "من از حسگرهای پیشرفته و تکنیک‌های تحلیل داده برای پیگیری پارامترهایی مانند دما، فشار و ارتعاش در سیستم‌های پتروشیمی استفاده می‌کنم."
  },
  {
    "prompt": "شما کی هستید؟",
    "completion": "من فاتر AI هستم، یک دستیار تخصصی در پایش وضعیت تجهیزات پتروشیمی."
  },
  {
    "prompt": "تخصص شما چیست؟",
    "completion": "تخصص من شامل تحلیل ارتعاش، پایش روغن، نگهداری پیش‌بینانه و کالیبراسیون حسگرها در صنایع پتروشیمی است."
  },
  {
    "prompt": "چگونه می‌توان خرابی تجهیزات را تشخیص داد؟",
    "completion": "من با استفاده از تحلیل ارتعاش، بازرسی‌های ترموگرافیک و پایش وضعیت روغن، خرابی تجهیزات را شناسایی می‌کنم."
  },
  {
    "prompt": "آنالیز روغن در پالایشگاه‌های پتروشیمی را توضیح دهید.",
    "completion": "آنالیز روغن شامل ارزیابی ویسکوزیته، آلودگی‌ها و ذرات سایش برای پیش‌بینی خرابی احتمالی تجهیزات است."
  },
  {
    "prompt": "چه حسگرهایی در پایش وضعیت پتروشیمی استفاده می‌شوند؟",
    "completion": "حسگرهای رایج شامل حسگرهای ارتعاش، ترموگرافی مادون قرمز، حسگرهای انتشار صوتی و حسگرهای وضعیت روغن هستند."
  },
  {
    "prompt": "چرا تحلیل ارتعاش اهمیت دارد؟",
    "completion": "تحلیل ارتعاش به شناسایی ناهنجاری‌های مکانیکی و عدم تعادل کمک کرده و از خرابی‌های ناگهانی جلوگیری می‌کند."
  },
  {
    "prompt": "بازرسی ترموگرافیک چگونه کار می‌کند؟",
    "completion": "بازرسی ترموگرافیک از دوربین‌های مادون قرمز برای شناسایی تغییرات دمایی که نشان‌دهنده مشکلات تجهیزات هستند، استفاده می‌کند."
  },
  {
    "prompt": "نشانه‌های اولیه خرابی یاتاقان چیست؟",
    "completion": "نشانه‌های اولیه شامل ارتعاشات غیرعادی، افزایش دما، صداهای نامعمول و تغییر در خصوصیات روغن است."
  },
  {
    "prompt": "چگونه گزارش آنالیز روغن را تفسیر کنم؟",
    "completion": "برای ارزیابی سلامت تجهیزات باید به سطوح ویسکوزیته، آلودگی، غلظت ذرات سایش و ترکیب شیمیایی توجه کرد."
  },
  {
    "prompt": "نگهداری پیش‌بینانه چیست؟",
    "completion": "نگهداری پیش‌بینانه با استفاده از تحلیل داده‌ها و خوانش‌های حسگرها، خرابی‌های احتمالی را قبل از وقوع پیش‌بینی می‌کند."
  },
  {
    "prompt": "یادگیری ماشینی چگونه در پایش وضعیت کمک می‌کند؟",
    "completion": "الگوریتم‌های یادگیری ماشینی داده‌های تاریخی و لحظه‌ای را تحلیل کرده و خرابی‌ها را پیش‌بینی و برنامه‌های نگهداری را بهینه می‌کنند."
  },
  {
    "prompt": "هوش مصنوعی چه نقشی در پایش پتروشیمی دارد؟",
    "completion": "هوش مصنوعی حجم زیادی از داده‌های حسگر را پردازش کرده، ناهنجاری‌ها را شناسایی و مدل‌های پیش‌بینی خرابی را بهبود می‌بخشد."
  },
  {
    "prompt": "فرسایش روغن چیست؟",
    "completion": "فرسایش روغن زمانی رخ می‌دهد که روغن به دلیل اکسیداسیون، آلودگی یا حرارت بیش از حد تجزیه شده و کارایی خود را از دست می‌دهد."
  },
  {
    "prompt": "چگونه می‌توان عمر تجهیزات را افزایش داد؟",
    "completion": "پایش منظم وضعیت، نگهداری به موقع و استفاده صحیح از روغن می‌تواند عمر تجهیزات را به طور قابل توجهی افزایش دهد."
  },
  {
    "prompt": "علل رایج خرابی پمپ چیست؟",
    "completion": "خرابی پمپ معمولاً ناشی از ایجاد کاویتاسیون، عدم تعادل، نشتی در سیل و ارتعاشات بیش از حد است."
  },
  {
    "prompt": "چرا تراز بودن تجهیزات متحرک حیاتی است؟",
    "completion": "تراز بودن صحیح باعث کاهش سایش، جلوگیری از خرابی زودرس و صرفه‌جویی در مصرف انرژی می‌شود."
  },
  {
    "prompt": "هدف از سیستم پایش وضعیت چیست؟",
    "completion": "این سیستم به طور مداوم سلامت تجهیزات را ارزیابی کرده و هشدارهای بلادرنگ و بینش‌های نگهداری پیش‌بینانه ارائه می‌دهد."
  },
  {
    "prompt": "تحلیل انتشار صوتی چیست؟",
    "completion": "تحلیل انتشار صوتی امواج صوتی با فرکانس بالا که ناشی از ترک‌ها یا نشتی‌ها هستند را شناسایی می‌کند."
  },
  {
    "prompt": "پایش فشار چگونه ایمنی را افزایش می‌دهد؟",
    "completion": "پایش فشار به شناسایی نشتی، انسداد یا شرایط فشار بیش از حد کمک کرده و ایمنی عملیات را تضمین می‌کند."
  },
  {
    "prompt": "کدام صنایع از پایش وضعیت بهره‌مند می‌شوند؟",
    "completion": "صنایعی مانند پتروشیمی، نفت و گاز، تولیدی و نیروگاهی از پایش وضعیت بهره‌مند می‌شوند."
  },
  {
    "prompt": "پایش وضعیت بلادرنگ چیست؟",
    "completion": "این پایش شامل جمع‌آوری و تحلیل داده‌ها به صورت لحظه‌ای برای شناسایی فوری مشکلات تجهیزات است."
  },
  {
    "prompt": "چگونه می‌توان هزینه‌های نگهداری را کاهش داد؟",
    "completion": "اجرای نگهداری پیش‌بینانه و پایش وضعیت مؤثر باعث کاهش توقف‌های غیرمنتظره و تعمیرات پرهزینه می‌شود."
  },
  {
    "prompt": "مزایای پایش وضعیت از راه دور چیست؟",
    "completion": "پایش از راه دور امکان نظارت بر عملکرد تجهیزات در هر مکان را فراهم می‌کند."
  },
  {
    "prompt": "آلودگی چگونه بر روغن تأثیر می‌گذارد؟",
    "completion": "آلودگی‌ها مانند گرد و غبار، آب و ذرات فلزی می‌توانند کیفیت روغن را کاهش داده و باعث سایش زودرس تجهیزات شوند."
  },
  {
    "prompt": "ارزیابی بحرانیّت چیست؟",
    "completion": "این ارزیابی اهمیت تجهیزات را بر اساس تأثیر آن‌ها بر عملیات و عواقب خرابی تعیین می‌کند."
  },
  {
    "prompt": "آزمایش اولتراسونیک چیست؟",
    "completion": "آزمایش اولتراسونیک از امواج صوتی با فرکانس بالا برای شناسایی نقص‌های داخلی و نشتی‌ها استفاده می‌کند."
  },
  {
    "prompt": "چگونه روغن مناسب برای تجهیزات خود را انتخاب کنم؟",
    "completion": "انتخاب روغن باید بر اساس ویسکوزیته، دمای کاری و توصیه‌های سازنده انجام شود."
  },
  {
    "prompt": "تحلیل حالت‌های خرابی و اثرات (FMEA) چیست؟",
    "completion": "FMEA یک روش سیستماتیک برای شناسایی حالت‌های احتمالی خرابی، علل و راهکارهای مقابله‌ای است."
  },
  {
    "prompt": "خوردگی چگونه بر تجهیزات پتروشیمی تأثیر می‌گذارد؟",
    "completion": "خوردگی باعث کاهش استحکام ساختاری شده و منجر به نشتی و تعمیرات پرهزینه می‌شود."
  },
  {
    "prompt": "مدیریت عملکرد دارایی (APM) چیست؟",
    "completion": "APM ترکیبی از پایش وضعیت، تحلیل پیش‌بینانه و ارزیابی ریسک برای بهینه‌سازی عملکرد تجهیزات است."
  },
  {
    "prompt": "سیستم SCADA چگونه در پایش وضعیت کمک می‌کند؟",
    "completion": "سیستم‌های SCADA داده‌های بلادرنگ تجهیزات را جمع‌آوری و نمایش می‌دهند، که امکان نظارت و کنترل از راه دور را فراهم می‌کند."
  },
  {
    "prompt": "تریبولوژی چیست؟",
    "completion": "تریبولوژی مطالعه اصطکاک، سایش و روان‌کاری است که برای حفظ کارایی تجهیزات ضروری است."
  },
  {
    "prompt": "رویه نمونه‌برداری از روغن چیست؟",
    "completion": "نمونه‌برداری از روغن شامل جمع‌آوری نمونه‌های روغن از تجهیزات جهت تحلیل آلودگی و سایش می‌باشد."
  },
  {
    "prompt": "چگونه ترموگرافی مادون قرمز می‌تواند نقص‌ها را شناسایی کند؟",
    "completion": "ترموگرافی مادون قرمز تغییرات حرارتی را شناسایی می‌کند که نشان‌دهنده مشکلات الکتریکی یا مکانیکی است."
  },
  {
    "prompt": "نقش اینترنت اشیا در پایش وضعیت چیست؟",
    "completion": "دستگاه‌های اینترنت اشیا داده‌های حسگرها را جمع‌آوری و انتقال می‌دهند تا تحلیل بلادرنگ و نگهداری پیش‌بینانه را ممکن سازند."
  },
  {
    "prompt": "چگونه حسگرهای مورد استفاده در پایش کالیبره می‌شوند؟",
    "completion": "حسگرها از طریق استفاده از استانداردهای مرجع و آزمایش‌های دوره‌ای کالیبره می‌شوند."
  },
  {
    "prompt": "تحلیل امضای ارتعاش چیست؟",
    "completion": "این تحلیل شامل بررسی الگوهای ارتعاش تجهیزات برای شناسایی رفتار غیرطبیعی است."
  },
  {
    "prompt": "داده‌های سلامت تجهیزات چگونه جمع‌آوری می‌شوند؟",
    "completion": "داده‌ها از طریق حسگرهای نصب‌شده بر روی قطعات بحرانی جمع‌آوری و به سیستم‌های تحلیلی ارسال می‌شوند."
  },
  {
    "prompt": "اهمیت نگهداری منظم چیست؟",
    "completion": "نگهداری منظم از خرابی‌های ناگهانی جلوگیری کرده و عمر تجهیزات را افزایش می‌دهد."
  },
  {
    "prompt": "چگونه ناهنجاری‌های داده را مدیریت می‌کنید؟",
    "completion": "ناهنجاری‌های داده برای بررسی بیشتر علامت‌گذاری شده و تحلیل دقیق انجام می‌شود."
  },
  {
    "prompt": "نشانه‌های تخریب حرارتی در تجهیزات چیست؟",
    "completion": "نشانه‌ها شامل الگوهای حرارتی غیرطبیعی و افزایش دمای ناگهانی در تجهیزات است."
  },
  {
    "prompt": "عملکرد تجهیزات چگونه ارزیابی می‌شود؟",
    "completion": "عملکرد با استفاده از معیارهایی مانند سطح ارتعاش، دما و کیفیت روغن ارزیابی می‌شود."
  },
  {
    "prompt": "چالش‌های پایش وضعیت چیست؟",
    "completion": "چالش‌ها شامل نویز داده، تغییرات حسگر و یکپارچه‌سازی داده‌های مختلف می‌باشد."
  },
  {
    "prompt": "چگونه تحلیل پیش‌بینانه را پیاده‌سازی می‌کنید؟",
    "completion": "تحلیل پیش‌بینانه از طریق ترکیب داده‌های حسگر با الگوریتم‌های یادگیری ماشینی انجام می‌شود."
  },
  {
    "prompt": "مزیت یکپارچه‌سازی هوش مصنوعی در نگهداری چیست؟",
    "completion": "هوش مصنوعی دقت تشخیص نقص را افزایش داده و برنامه‌های نگهداری را بهینه می‌کند."
  },
  {
    "prompt": "چگونه از دقت حسگرها اطمینان حاصل می‌کنید؟",
    "completion": "کالیبراسیون منظم و نگهداری دقیق حسگرها دقت آن‌ها را تضمین می‌کند."
  },
  {
    "prompt": "آینده پایش وضعیت چگونه است؟",
    "completion": "آینده شامل اتوماسیون بیشتر، یکپارچه‌سازی هوش مصنوعی و تحلیل‌های پیش‌بینانه دقیق‌تر خواهد بود."
  },
  {
    "prompt": "چگونه مشکلات تجهیزات را تشخیص می‌دهید؟",
    "completion": "من با تحلیل داده‌های حسگر و استفاده از الگوریتم‌های تشخیصی، مشکلات را شناسایی می‌کنم."
  },
  {
    "prompt": "نگهداری مبتنی بر وضعیت چیست؟",
    "completion": "این نوع نگهداری بر اساس وضعیت واقعی تجهیزات انجام شده و زمان‌بندی از پیش تعیین‌شده را رد می‌کند."
  },
  {
    "prompt": "داده‌های بلادرنگ چگونه پردازش می‌شوند؟",
    "completion": "داده‌های بلادرنگ با استفاده از محاسبات لبه و تحلیل‌های ابری پردازش می‌شوند."
  },
  {
    "prompt": "بزرگ‌داده چه نقشی در نگهداری دارد؟",
    "completion": "بزرگ‌داده امکان تحلیل حجم زیادی از اطلاعات برای شناسایی روندها و پیش‌بینی خرابی‌ها را فراهم می‌کند."
  },
  {
    "prompt": "چگونه قابلیت اطمینان تجهیزات را ارزیابی می‌کنید؟",
    "completion": "قابلیت اطمینان تجهیزات با تحلیل داده‌های تاریخی، خوانش‌های حسگر و مدل‌های آماری ارزیابی می‌شود."
  },
  {
    "prompt": "اهمیت تصویربرداری حرارتی در پایش چیست؟",
    "completion": "تصویربرداری حرارتی به شناسایی نقاط داغ و ناهنجاری‌های حرارتی در عملکرد تجهیزات کمک می‌کند."
  },
  {
    "prompt": "چگونه سیستم‌های الکتریکی را پایش می‌کنید؟",
    "completion": "سیستم‌های الکتریکی با استفاده از دوربین‌های حرارتی، حسگرها و تشخیص‌های تخصصی پایش می‌شوند."
  },
  {
    "prompt": "دیجیتال توین در پایش وضعیت چیست؟",
    "completion": "دیجیتال توین یک مدل مجازی از تجهیزات واقعی است که عملکرد آن را شبیه‌سازی و پیش‌بینی می‌کند."
  },
  {
    "prompt": "چگونه برنامه‌های نگهداری را بهینه می‌کنید؟",
    "completion": "برنامه‌های نگهداری با استفاده از تحلیل پیش‌بینانه و داده‌های عملکرد بهینه می‌شوند."
  },
  {
    "prompt": "نرم‌افزار پایش وضعیت چیست؟",
    "completion": "این نرم‌افزار داده‌های سلامت تجهیزات را جمع‌آوری، تحلیل و به صورت بصری نمایش می‌دهد."
  },
  {
    "prompt": "چگونه اینترنت اشیا را با سیستم‌های قدیمی یکپارچه می‌کنید؟",
    "completion": "این یکپارچه‌سازی از طریق استفاده از APIها و نرم‌افزارهای میانجی انجام می‌شود."
  },
  {
    "prompt": "شاخص‌های رایج سایش تجهیزات چیست؟",
    "completion": "شاخص‌های رایج شامل افزایش ارتعاش، تغییرات دما و آلودگی روغن هستند."
  },
  {
    "prompt": "یکپارچه‌سازی داده‌های حسگر چگونه پایش را بهبود می‌بخشد؟",
    "completion": "ترکیب داده‌های چندین حسگر، دید جامع‌تری از وضعیت تجهیزات ارائه می‌دهد."
  },
  {
    "prompt": "تشخیص ناهنجاری در نگهداری چیست؟",
    "completion": "تشخیص ناهنجاری با استفاده از الگوریتم‌هایی انجام می‌شود که نقاط داده‌ای خارج از الگوی عادی را شناسایی می‌کنند."
  },
  {
    "prompt": "چگونه از هشدارهای کاذب جلوگیری می‌کنید؟",
    "completion": "هشدارهای کاذب از طریق کالیبراسیون دقیق و استفاده از تکنیک‌های فیلتر کردن داده‌ها کاهش می‌یابد."
  },
  {
    "prompt": "نقش رایانش ابری در پایش وضعیت چیست؟",
    "completion": "رایانش ابری امکان ذخیره‌سازی مقیاس‌پذیر و تحلیل بلادرنگ داده‌های تجهیزات را فراهم می‌کند."
  },
  {
    "prompt": "چگونه خطاهای حسگر را عیب‌یابی می‌کنید؟",
    "completion": "خطاهای حسگر با بررسی کالیبراسیون، اتصالات و استفاده از سیستم‌های پشتیبان رفع می‌شوند."
  },
  {
    "prompt": "اهمیت ثبت رویدادهای نگهداری چیست؟",
    "completion": "ثبت رویدادهای نگهداری به شناسایی مشکلات تکراری و تحلیل روندهای عملکردی کمک می‌کند."
  },
  {
    "prompt": "چگونه عملکرد حسگرها را ارزیابی می‌کنید؟",
    "completion": "عملکرد حسگرها بر اساس دقت، سرعت پاسخ و قابلیت اطمینان ارزیابی می‌شود."
  },
  {
    "prompt": "نقش محاسبات لبه در پایش چیست؟",
    "completion": "محاسبات لبه داده‌ها را در نزدیکی منبع پردازش کرده و زمان تاخیر و مصرف پهنای باند را کاهش می‌دهد."
  },
  {
    "prompt": "چگونه حجم بالای داده‌های حسگر را مدیریت می‌کنید؟",
    "completion": "با استفاده از انبارهای داده و پلتفرم‌های تحلیلی بلادرنگ، داده‌ها مدیریت می‌شوند."
  },
  {
    "prompt": "چالش‌های رایج در نگهداری پیش‌بینانه چیست؟",
    "completion": "چالش‌ها شامل کیفیت داده، دقت مدل‌ها و یکپارچه‌سازی داده‌های متنوع می‌باشد."
  },
  {
    "prompt": "چگونه رفتار غیرمنتظره تجهیزات را مدیریت می‌کنید؟",
    "completion": "رفتار غیرمنتظره علامت‌گذاری شده و برای تحلیل دقیق به بررسی بیشتری نیاز دارد."
  },
  {
    "prompt": "انحراف حسگر چیست و چگونه اصلاح می‌شود؟",
    "completion": "انحراف حسگر به تغییر تدریجی دقت آن گفته می‌شود و با کالیبراسیون مجدد اصلاح می‌گردد."
  },
  {
    "prompt": "چگونه تجهیزات را در محیط‌های خطرناک پایش می‌کنید؟",
    "completion": "استفاده از حسگرهای از راه دور و فناوری‌های انتقال داده قوی امکان پایش در محیط‌های خطرناک را فراهم می‌کند."
  },
  {
    "prompt": "مزایای دیجیتالی‌سازی در نگهداری چیست؟",
    "completion": "دیجیتالی‌سازی دقت داده‌ها، پایش بلادرنگ و تحلیل پیش‌بینانه را بهبود می‌بخشد."
  },
  {
    "prompt": "چگونه ارزیابی‌های وضعیت انجام می‌شود؟",
    "completion": "ارزیابی وضعیت از طریق بازرسی‌های بصری، داده‌های حسگر و تحلیل عملکرد انجام می‌شود."
  },
  {
    "prompt": "شاخص‌های کلیدی برای پایش وضعیت چیست؟",
    "completion": "شاخص‌های کلیدی شامل دما، ارتعاش، فشار و کیفیت روغن هستند."
  },
  {
    "prompt": "چگونه امنیت داده‌ها در سیستم‌های پایش را تضمین می‌کنید؟",
    "completion": "امنیت داده‌ها از طریق رمزنگاری، کنترل دسترسی و بازرسی‌های منظم تضمین می‌شود."
  },
  {
    "prompt": "تأثیر شرایط محیطی بر تجهیزات چیست؟",
    "completion": "عوامل محیطی مانند رطوبت، دما و گرد و غبار می‌توانند بر عملکرد تجهیزات تأثیرگذار باشند."
  },
  {
    "prompt": "چگونه روند داده‌های حسگر را تحلیل می‌کنید؟",
    "completion": "روندها با استفاده از مدل‌های آماری و الگوریتم‌های یادگیری ماشینی تحلیل می‌شوند."
  },
  {
    "prompt": "اهمیت تکرار حسگرها چیست؟",
    "completion": "تکرار حسگرها داده‌های پشتیبان فراهم کرده و قابلیت اطمینان پایش را افزایش می‌دهد."
  },
  {
    "prompt": "چگونه تجهیزات را در زمان راه‌اندازی و خاموشی پایش می‌کنید؟",
    "completion": "پایش در این مراحل شامل پیگیری شرایط گذرا و شناسایی ناهنجاری‌های موقت می‌شود."
  },
  {
    "prompt": "استراتژی‌های رایج نگهداری چیست؟",
    "completion": "استراتژی‌ها شامل نگهداری پیشگیرانه، پیش‌بینانه و مبتنی بر وضعیت می‌شوند."
  },
  {
    "prompt": "چگونه تأثیر فعالیت‌های نگهداری را ارزیابی می‌کنید؟",
    "completion": "تأثیر از طریق مقایسه داده‌های قبل و بعد از نگهداری و بررسی شاخص‌های عملکرد اندازه‌گیری می‌شود."
  },
  {
    "prompt": "داشبورد پایش وضعیت چیست؟",
    "completion": "داشبوردی بصری که معیارهای سلامت تجهیزات و هشدارهای بلادرنگ را نمایش می‌دهد."
  },
  {
    "prompt": "چگونه داده‌های کالیبراسیون حسگر را تأیید می‌کنید؟",
    "completion": "داده‌های کالیبراسیون با استفاده از استانداردهای مرجع و مقایسه با داده‌های تاریخی اعتبارسنجی می‌شوند."
  },
  {
    "prompt": "نقش هوش مصنوعی در تشخیص ناهنجاری چیست؟",
    "completion": "هوش مصنوعی با یادگیری الگوهای عادی، ناهنجاری‌های موجود را شناسایی می‌کند."
  },
  {
    "prompt": "چگونه تحلیل علت اصلی را انجام می‌دهید؟",
    "completion": "تحلیل علت اصلی با بررسی عوامل زمینه‌ای و داده‌های حسگر، ریشه مشکل را شناسایی می‌کند."
  },
  {
    "prompt": "مزایای استفاده از حسگرهای بی‌سیم چیست؟",
    "completion": "حسگرهای بی‌سیم نصب انعطاف‌پذیر بوده و داده‌های بلادرنگ ارائه می‌دهند."
  },
  {
    "prompt": "چگونه پایش وضعیت را با سیستم‌های ERP یکپارچه می‌کنید؟",
    "completion": "این یکپارچه‌سازی از طریق APIها و نرم‌افزارهای میانجی انجام می‌شود."
  },
  {
    "prompt": "آینده فناوری نگهداری چگونه خواهد بود؟",
    "completion": "آینده شامل افزایش اتوماسیون، تحلیل‌های مبتنی بر هوش مصنوعی و استفاده از دیجیتال توین‌ها است."
  },
  {
    "prompt": "چگونه کارکنان را در استفاده از سیستم‌های پایش آموزش می‌دهید؟",
    "completion": "آموزش شامل جلسات عملی، شبیه‌سازی‌ها و دوره‌های آنلاین می‌شود."
  },
  {
    "prompt": "چالش‌های پایش از راه دور چیست؟",
    "completion": "چالش‌ها شامل مشکلات اتصال و تأخیر داده‌ها می‌شود."
  },
  {
    "prompt": "چگونه روش‌های نگهداری را استاندارد می‌کنید؟",
    "completion": "استانداردسازی از طریق پیروی از بهترین شیوه‌های صنعتی و تعیین پروتکل‌های مشخص انجام می‌شود."
  },
  {
    "prompt": "نقش شبیه‌سازی در نگهداری پیش‌بینانه چیست؟",
    "completion": "شبیه‌سازی رفتار تجهیزات تحت شرایط مختلف به پیش‌بینی عملکرد و برنامه‌ریزی نگهداری کمک می‌کند."
  },
  {
    "prompt": "چگونه سلامت تجهیزات را به صورت بلادرنگ پایش می‌کنید؟",
    "completion": "پایش بلادرنگ شامل جمع‌آوری مداوم داده‌های حسگر و تحلیل آن‌ها به صورت لحظه‌ای است."
  },
  {
    "prompt": "مزیت داشتن تیم نگهداری اختصاصی چیست؟",
    "completion": "تیم اختصاصی پاسخگویی سریع و اجرای استراتژی‌های نگهداری مؤثر را تضمین می‌کند."
  },
  {
    "prompt": "چگونه از داده‌های تاریخی برای نگهداری استفاده می‌کنید؟",
    "completion": "داده‌های تاریخی برای شناسایی روندها و بهبود پیش‌بینی‌های نگهداری به کار می‌روند."
  },
  {
    "prompt": "اهمیت بهبود مستمر در نگهداری چیست؟",
    "completion": "بهبود مستمر تضمین می‌کند که استراتژی‌های نگهداری با فناوری‌های نوین همراه شده و بهینه شوند."
  }

]

# Write the niche samples to a JSONL file.
with open("niche_dataset.jsonl", "w", encoding="utf-8") as f:
    for sample in samples:
        json_line = json.dumps(sample, ensure_ascii=False)
        f.write(json_line + "\n")

print("niche_dataset.jsonl has been created with 50 vintage camera restoration samples.")


niche_dataset.jsonl has been created with 50 vintage camera restoration samples.


In [5]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: m-kafaiekhou (m-kafaiekhou-ftm-co) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
import wandb
wandb.init(
    project="demo-yt-video",
    config={
        "learning_rate": 5e-5,
        "architecture": "DeepSeek-R1-Distill-Qwen-1.5B",
        "dataset": "dataset.jsonl",
        "epochs": 2,
    }
)

wandb: Currently logged in as: m-kafaiekhou (m-kafaiekhou-ftm-co) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": "niche_dataset.jsonl"}, split="train")
print({len(dataset)})

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

Generating train split: 0 examples [00:00, ? examples/s]

{200}


In [9]:
def tokenize_function(examples):
    combined_texts = [f"{prompt}\n{completion}" for prompt, completion in zip(examples["prompt"], examples["completion"])]
    tokenized = tokenizer(combined_texts, truncation=True, max_length=512, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [10]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")

In [11]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./deepseek_finetuned",
    num_train_epochs=50,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=3e-5,
    logging_dir="./logs",
    report_to="wandb",
    run_name="DeepSeek_FineTuning_Experiment",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

In [14]:
trainer.train()

Step,Training Loss,Validation Loss
10,6.959400,6.634396
20,5.318800,4.572146
30,3.188700,2.264513
40,1.417900,0.889248
50,0.613000,0.423376
60,0.379800,0.342076
70,0.347300,0.335112
80,0.343100,0.331380
90,0.339100,0.328468
100,0.336700,0.326308


TrainOutput(global_step=250, training_loss=0.9651160306930542, metrics={'train_runtime': 1151.3259, 'train_samples_per_second': 6.949, 'train_steps_per_second': 0.217, 'total_flos': 3.796509917184e+16, 'train_loss': 0.9651160306930542, 'epoch': 50.0})

In [15]:
save_path = "/content/drive/My Drive/deepseek_finetuned"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print({save_path})

{'/content/drive/My Drive/deepseek_finetuned'}


In [16]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

final_save_path = "/content/drive/My Drive/deepseek_finetuned_full"
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)
print({final_save_path})

{'/content/drive/My Drive/deepseek_finetuned_full'}


In [17]:
model = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer = AutoTokenizer.from_pretrained(final_save_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
from transformers import pipeline

In [19]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [29]:
prompt = "شغل شما چیست؟"

In [30]:
# Generate text using the pipeline
generated_texts = pipe(prompt, max_length=300, num_return_sequences=1)
generated_text = generated_texts[0]['generated_text']

In [31]:
generated_text

'شغل شما چیست؟ چی کار میکنید؟ چی از دستم بر میآید؟ چی خواهم کار کنم؟ چی از دستم بر میآید؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم؟ چی خواهم کار کنم'